In [1]:
import io
import os
import numpy as np

from typing import Dict, List, Optional, OrderedDict, Tuple

from datasets import load_dataset, Features, ClassLabel, Image

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.utilities.seed import seed_everything

import torch
import torch.nn as nn

from torchvision import datasets, transforms
from torchvision.utils import save_image

from fid_score import calculate_fretchet
from inception import InceptionV3


In [2]:
ALL_IMAGES = []

img_size = 128
batch_size = 32
normalize = [(0.5), (0.5)]
latent_size = 256
data_dir = "ChainYo/rvl-cdip-invoice"

In [3]:
transforms = transforms.Compose(
    [
        transforms.Resize(img_size), 
        transforms.CenterCrop(img_size), 
        transforms.ToTensor(), 
        # transforms.Normalize(*normalize)
    ]
)

In [4]:
dataset = load_dataset(data_dir, split="train")
print(dataset)

labels = dataset.features["label"].names
num_labels = dataset.features["label"].num_classes


def preprocess_data(examples):
    examples["tr_image"] = [transforms(img) for img in examples["image"]]
    del examples["image"]
    return examples


transformed_dataset = dataset.with_transform(preprocess_data)


Using custom data configuration ChainYo--rvl-cdip-invoice-e89502dfe52b2353
Reusing dataset parquet (/home/chainyo/.cache/huggingface/datasets/parquet/ChainYo--rvl-cdip-invoice-e89502dfe52b2353/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Dataset({
    features: ['image', 'label'],
    num_rows: 19947
})


In [5]:
train_dataloader = torch.utils.data.DataLoader(
    transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
)

In [6]:
def denormalize(input_image_tensors: torch.Tensor) -> torch.Tensor:
    """
    Denormalizes the input image tensors.

    Parameters
    ----------
    input_image_tensors : torch.Tensor
        The input image tensors.
    
    Returns
    -------
    torch.Tensor
        The denormalized image tensors.
    """
    return input_image_tensors.mul(normalize[1]).add(normalize[0])


def save_samples(index: int, sample_images: torch.Tensor) -> None:
    """
    Saves the generated samples.

    Parameters
    ----------
    index : int
        The index of the sample.
    sample_images : torch.Tensor
        The generated sample images.
    """
    fake_name = f"generated-images-{index}.png"
    # save_image(denormalize(sample_images[-64:]), os.path.join("generated", fake_name), nrow=8)
    save_image(sample_images[-64:], os.path.join("generated", fake_name), nrow=8)

In [7]:
class Discriminator(nn.Module):
    def __init__(
        self,
        hidden_size: Optional[int] = 64,
        channels: Optional[int] = 3,
        kernel_size: Optional[int] = 4,
        stride: Optional[int] = 2,
        padding: Optional[int] = 1,
        negative_slope: Optional[float] = 0.2,
        bias: Optional[bool] = False,
    ):
        """
        Initializes the discriminator.

        Parameters
        ----------
        hidden_size : int, optional
            The input size. (the default is 64)
        channels : int, optional
            The number of channels. (default: 3)
        kernel_size : int, optional
            The kernal size. (default: 4)
        stride : int, optional
            The stride. (default: 2)
        padding : int, optional
            The padding. (default: 1)
        negative_slope : float, optional
            The negative slope. (default: 0.2)
        bias : bool, optional
            Whether to use bias. (default: False)
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.channels = channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.negative_slope = negative_slope
        self.bias = bias

        self.model = nn.Sequential(
            # input size: (channels, 64, 64)
            nn.utils.spectral_norm(
                nn.Conv2d(
                    self.channels, self.hidden_size, 
                    kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
                ),
            ),
            nn.BatchNorm2d(self.hidden_size),
            nn.LeakyReLU(self.negative_slope, inplace=True),

            # input size: (hidden_size, 32, 32)
            nn.utils.spectral_norm(
                nn.Conv2d(
                    hidden_size, hidden_size * 2, 
                    kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
                ),
            ),
            nn.BatchNorm2d(hidden_size * 2),
            nn.LeakyReLU(self.negative_slope, inplace=True),

            # input size: (hidden_size * 2, 16, 16)
            nn.utils.spectral_norm(
                nn.Conv2d(
                    hidden_size * 2, hidden_size * 4,
                    kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
                ),
            ),
            nn.BatchNorm2d(hidden_size * 4),
            nn.LeakyReLU(self.negative_slope, inplace=True),

            # input size: (hidden_size * 4, 8, 8)
            nn.utils.spectral_norm(
                nn.Conv2d(
                    hidden_size * 4, hidden_size * 8, 
                    kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
                ),
            ),
            nn.BatchNorm2d(hidden_size * 8),
            nn.LeakyReLU(self.negative_slope, inplace=True),

            # input size: (hidden_size * 8, 4, 4)
            nn.utils.spectral_norm(
                nn.Conv2d(hidden_size * 8, 1, kernel_size=4, stride=1, padding=0, bias=self.bias), # output size: (1, 1, 1)
            ),
            nn.Flatten(),
        )

    
    def forward(self, input_img: torch.Tensor) -> torch.Tensor:
        """
        Forward propagation.

        Parameters
        ----------
        input_img : torch.Tensor
            The input image.

        Returns
        -------
        torch.Tensor
            The output.
        """
        logits = self.model(input_img)
        return logits

In [8]:
class Generator(nn.Module):
    def __init__(
        self,
        hidden_size: Optional[int] = 64,
        latent_size: Optional[int] = 128,
        channels: Optional[int] = 3,
        kernel_size: Optional[int] = 4,
        stride: Optional[int] = 2,
        padding: Optional[int] = 1,
        bias: Optional[bool] = False,
    ):
        """
        Initializes the generator.

        Parameters
        ----------
        hidden_size : int, optional
            The hidden size. (default: 64)
        latent_size : int, optional
            The latent size. (default: 128)
        channels : int, optional
            The number of channels. (default: 3)
        kernel_size : int, optional
            The kernel size. (default: 4)
        stride : int, optional
            The stride. (default: 2)
        padding : int, optional
            The padding. (default: 1)
        bias : bool, optional
            Whether to use bias. (default: False)
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.channels = channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bias = bias

        self.model = nn.Sequential(
            # input size: (latent_size=128, 1, 1)
            nn.ConvTranspose2d(
                self.latent_size, self.hidden_size * 8, 
                kernel_size=self.kernel_size, stride=1, padding=0, bias=self.bias
            ),
            nn.BatchNorm2d(self.hidden_size * 8),
            nn.ReLU(inplace=True),

            # input size: (hidden_size * 8, 4, 4)
            nn.ConvTranspose2d(
                self.hidden_size * 8, self.hidden_size * 4, 
                kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
            ),
            nn.BatchNorm2d(self.hidden_size * 4),
            nn.ReLU(inplace=True),

            # input size: (hidden_size * 4, 8, 8)
            nn.ConvTranspose2d(
                self.hidden_size * 4, self.hidden_size * 2, 
                kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
            ),
            nn.BatchNorm2d(self.hidden_size * 2),
            nn.ReLU(inplace=True),

            # input size: (self.hidden_size * 2, 16, 16)
            nn.ConvTranspose2d(
                self.hidden_size * 2, self.hidden_size, 
                kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
            ),
            nn.BatchNorm2d(self.hidden_size),
            nn.ReLU(inplace=True),

            # input size: (64, 32, 32)
            nn.ConvTranspose2d(
                self.hidden_size, self.channels, 
                kernel_size=self.kernel_size, stride=self.stride, padding=self.padding, bias=self.bias
            ),
            nn.Tanh() # output size: (channels, 64, 64)
        )

    
    def forward(self, input_noise: torch.Tensor) -> torch.Tensor:
        """
        Forward propagation.

        Parameters
        ----------
        input_noise : torch.Tensor
            The input image.

        Returns
        -------
        torch.Tensor
            The output.
        """
        fake_img = self.model(input_noise)
        return fake_img

In [9]:
class DocuGAN(pl.LightningModule):
    def __init__(
        self,
        hidden_size: Optional[int] = 64,
        latent_size: Optional[int] = 128,
        num_channel: Optional[int] = 3,
        learning_rate: Optional[float] = 0.0002,
        batch_size: Optional[int] = 128,
        bias1: Optional[float] = 0.5,
        bias2: Optional[float] = 0.999,
    ):
        """
        Initializes the LightningGan.

        Parameters
        ----------
        hidden_size : int, optional
            The hidden size. (default: 64)
        latent_size : int, optional
            The latent size. (default: 128)
        num_channel : int, optional
            The number of channels. (default: 3)
        learning_rate : float, optional
            The learning rate. (default: 0.0002)
        batch_size : int, optional
            The batch size. (default: 128)
        bias1 : float, optional
            The bias1. (default: 0.5)
        bias2 : float, optional
            The bias2. (default: 0.999)
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.num_channel = num_channel
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.bias1 = bias1
        self.bias2 = bias2
        self.criterion = nn.BCEWithLogitsLoss()
        self.validation = torch.randn(self.batch_size, self.latent_size, 1, 1)
        self.save_hyperparameters()

        self.generator = Generator(
            latent_size=self.latent_size, channels=self.num_channel, hidden_size=self.hidden_size
        )
        self.generator.apply(self.weights_init)
        
        self.discriminator = Discriminator(channels=self.num_channel, hidden_size=self.hidden_size)
        self.discriminator.apply(self.weights_init)

        # self.model = InceptionV3() # For FID metric


    def weights_init(self, m: nn.Module) -> None:
        """
        Initializes the weights.

        Parameters
        ----------
        m : nn.Module
            The module.
        """
        classname = m.__class__.__name__
        if classname.find("Conv") != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find("BatchNorm") != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)

    
    def configure_optimizers(self) -> Tuple[List[torch.optim.Optimizer], List]:
        """
        Configures the optimizers.

        Returns
        -------
        Tuple[List[torch.optim.Optimizer], List]
            The optimizers and the LR schedulers.
        """
        opt_generator = torch.optim.Adam(
            self.generator.parameters(), lr=self.learning_rate, betas=(self.bias1, self.bias2)
        )
        opt_discriminator = torch.optim.Adam(
            self.discriminator.parameters(), lr=self.learning_rate, betas=(self.bias1, self.bias2)
        )
        return [opt_generator, opt_discriminator], []

    
    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """
        Forward propagation.

        Parameters
        ----------
        z : torch.Tensorh
            The latent vector.

        Returns
        -------
        torch.Tensor
            The output.
        """
        return self.generator(z)

    
    def training_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int, optimizer_idx: int
    ) -> Dict:
        """
        Training step.

        Parameters
        ----------
        batch : Tuple[torch.Tensor, torch.Tensor]
            The batch.
        batch_idx : int
            The batch index.
        optimizer_idx : int
            The optimizer index.

        Returns
        -------
        Dict
            The training loss.
        """
        real_images = batch["tr_image"]

        if optimizer_idx == 0: # Only train the generator
            fake_random_noise = torch.randn(self.batch_size, self.latent_size, 1, 1)
            fake_random_noise = fake_random_noise.type_as(real_images)
            fake_images = self(fake_random_noise)

            # Try to fool the discriminator
            preds = self.discriminator(fake_images)
            loss = self.criterion(preds, torch.ones_like(preds))
            self.log("g_loss", loss, on_step=False, on_epoch=True)

            if self.num_channel == 3:
                fid = calculate_fretchet(real_images, fake_images, self.model.to(self.device))
                self.log("fid", fid, on_step=False, on_epoch=True)

            tqdm_dict = {"g_loss": loss}
            output = OrderedDict({"loss": loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

        elif optimizer_idx == 1: # Only train the discriminator
            real_preds = self.discriminator(real_images)
            real_loss = self.criterion(real_preds, torch.ones_like(real_preds))

            # Generate fake images
            real_random_noise = torch.randn(self.batch_size, self.latent_size, 1, 1)
            real_random_noise = real_random_noise.type_as(real_images)
            fake_images = self(real_random_noise)

            # Pass fake images though discriminator
            fake_preds = self.discriminator(fake_images)
            fake_loss = self.criterion(fake_preds, torch.zeros_like(fake_preds))

            # Update discriminator weights
            loss = real_loss + fake_loss
            self.log("d_loss", loss, on_step=False, on_epoch=True)

            tqdm_dict = {"d_loss": loss}
            output = OrderedDict({"loss": loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

    
    def on_epoch_end(self):
        """
        Called at the end of an epoch.
        """
        z = self.validation.type_as(self.generator.model[0].weight)
        sample_images = self(z)
        ALL_IMAGES.append(sample_images.detach().cpu())
        save_samples(self.current_epoch, sample_images)
        self.logger[1].log_image(key=f"images-epoch{self.current_epoch}", images=[sample_images])

In [10]:
seed_everything(42)
gpus = 1 if torch.cuda.is_available() else 0

tf_logger = TensorBoardLogger("logs", name="docugan")
wandb_logger = WandbLogger(project="docugan")

model = DocuGAN(hidden_size=img_size, num_channel=1, latent_size=latent_size, batch_size=batch_size)

trainer = pl.Trainer(
    gpus=gpus,
    max_epochs=500,
    progress_bar_refresh_rate=25,
    # callbacks=[early_stopping, checkpointer],
    logger=[tf_logger, wandb_logger],
)
trainer.fit(model, train_dataloader)

Global seed set to 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chainyo-mleng (use `wandb login --relogin` to force relogin)


/home/chainyo/miniconda3/envs/gan-bird/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=25)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/chainyo/miniconda3/envs/gan-bird/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:307: LightningDeprecationWarning: The `LightningModule.on_epoch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `LightningModule.on_<train/validation/test>_epoch_end` instead.
  rank_zero_deprecation(


Epoch 0: 100%|██████████| 624/624 [00:53<00:00, 11.56it/s, loss=4.31, v_num=jtgy]

/home/chainyo/miniconda3/envs/gan-bird/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2719: UserWarning: Using trainer.logger when Trainer is configured to use multiple loggers. This behavior will change in v1.8 when LoggerCollection is removed, and trainer.logger will return the first logger in trainer.loggers
  rank_zero_warn(


Epoch 1: 100%|██████████| 624/624 [01:48<00:00,  5.77it/s, loss=4.97, v_num=jtgy]

...

/home/chainyo/miniconda3/envs/gan-bird/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2719: UserWarning: Using trainer.logger when Trainer is configured to use multiple loggers. This behavior will change in v1.8 when LoggerCollection is removed, and trainer.logger will return the first logger in trainer.loggers
  rank_zero_warn(


Epoch 98: 100%|██████████| 624/624 [1:18:53<00:00,  7.59s/it, loss=13.4, v_num=jtgy]   

/home/chainyo/miniconda3/envs/gan-bird/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2719: UserWarning: Using trainer.logger when Trainer is configured to use multiple loggers. This behavior will change in v1.8 when LoggerCollection is removed, and trainer.logger will return the first logger in trainer.loggers
  rank_zero_warn(


Epoch 99:  20%|██        | 125/624 [1:19:06<5:15:46, 37.97s/it, loss=13.4, v_num=jtgy] 

/home/chainyo/miniconda3/envs/gan-bird/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
